<a href="https://colab.research.google.com/github/alexmjn/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-12-04 20:19:51--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.41.150
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.41.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  17.0MB/s    in 13s     

2019-12-04 20:20:05 (15.2 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
import numpy as np
import pandas as pd

order_products_prior = pd.read_csv("order_products__prior.csv")
order_products_train = pd.read_csv("order_products__train.csv")
# Since these two csvs represent disjoint sets with identical 
# features, we can merge them with simple concatenation.
order_products = pd.concat([order_products_prior, order_products_train])

In [8]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [10]:
# We can drop the features that don't affect our eventual output.
order_products = order_products.drop(["add_to_cart_order", "reordered"], axis = 1)
order_products.head()

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


In [11]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [13]:
orders = orders.drop(["order_number", "order_dow", 
                      "order_hour_of_day", "days_since_prior_order"], axis = 1)
orders.head()

,order_id,user_id,eval_set
0,2539329,1,prior
1,2398795,1,prior
2,473747,1,prior
3,2254736,1,prior
4,431534,1,prior


In [0]:
orders = orders.drop(["eval_set"], axis = 1)

In [15]:
products = pd.read_csv("products.csv")
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [17]:
products = products.drop(["aisle_id", "department_id"], axis = 1)
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


We need to tie the product name to the actual list of orders made. To do this, we need "order_number" (this is the list of unique items ordered) from **product_order**, "product_name" from **products**, and a column to link these two databases. In this case, that column is "column_id." After loading in the **orders** data frame, it turns out to not be necessary for this problem.

In [22]:
# As order_products lists every individual product order,
# we can use value_counts() to derive the product ids.

most_frequent_product_ids = order_products["product_id"].value_counts()
most_frequent_product_ids = most_frequent_product_ids.head(10)
most_frequent_product_ids

24852    491291
13176    394930
21137    275577
21903    251705
47209    220877
47766    184224
47626    160792
16797    149445
26209    146660
27845    142813
Name: product_id, dtype: int64

In [21]:
# Verifying that this information matches what we were given.
# (Note that we now have the product IDs and counts for each
# of the most frequent products and so, if we did not want to 
# actually perform merge, could just use this method to obtain the 
# product names.

products[products["product_id"] == 24852]

,product_id,product_name
24851,24852,Banana


In [28]:
products_subset = products[products["product_id"].isin([24852, 13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209, 27845])]
products_subset.head(10)

,product_id,product_name
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21136,21137,Organic Strawberries
21902,21903,Organic Baby Spinach
24851,24852,Banana
26208,26209,Limes
27844,27845,Organic Whole Milk
47208,47209,Organic Hass Avocado
47625,47626,Large Lemon
47765,47766,Organic Avocado


In [30]:
order_products_subset = order_products[order_products["product_id"].isin([24852, 13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209, 27845])]
order_products_subset.head()

,order_id,product_id
12,3,21903
30,5,13176
51,5,47209
77,10,24852
80,10,47766


In [0]:
# Now that the two data frames we need are trimmed, we can merge
# on the unique identifier "product_id" to link the orders to the 
# product names.

common_product_orders = pd.merge(products_subset, order_products_subset, 
                                 how = "inner",
                                 on = "product_id")

In [35]:
# Now we use this merged data to generate the final sums.
common_product_orders["product_name"].value_counts()

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [36]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [38]:
table2 = table2.reset_index()
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [41]:
tidy = table2.melt(id_vars = "index", value_vars = ["John Smith", "Jane Doe", "Mary Johnson"])
tidy

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [42]:
tidy.columns = ["treatment", "name", "result"]
tidy

,treatment,name,result
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [43]:
tidy["treatment"] = tidy["treatment"].str.replace("treatment","")
tidy

,treatment,name,result
0,a,John Smith,NaN
1,b,John Smith,2.0
2,a,Jane Doe,16.0
3,b,Jane Doe,11.0
4,a,Mary Johnson,3.0
5,b,Mary Johnson,1.0


In [44]:
wide_data = tidy.pivot_table(index = "treatment", columns = "name", values = "result")
wide_data

name,Jane Doe,John Smith,Mary Johnson
treatment,,,
a,16.0,NaN,3.0
b,11.0,2.0,1.0


Load seaborn's flights dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [46]:
 import seaborn as sns
 flights = sns.load_dataset('flights')
 flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [47]:
pivot = flights.pivot_table(values = "passengers", index = "year", columns = "month")
pivot

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [48]:
order_products.head()

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


In [68]:
# We only want products ordered at least 2,900 times. So 
# I start this filtering process by grabbing a data frame 
# that contains every product ID along with its order frequency.

ids = order_products["product_id"].value_counts()
ids = ids.to_frame()
ids.head()

,product_id
24852,491291
13176,394930
21137,275577
21903,251705
47209,220877


In [0]:
# Format the data frame logically.
ids = ids.reset_index()
ids.columns = ["product_id", "count"]


In [73]:
# Filter the data frame, then convert it to a list
# to pass in to future arguments.
condition = ids["count"] > 2900
ids = ids[condition]
ids.head()

,product_id,count
0,24852,491291
1,13176,394930
2,21137,275577
3,21903,251705
4,47209,220877


In [77]:
list_of_ids = ids["product_id"].tolist()
list_of_ids[:5]

[24852, 13176, 21137, 21903, 47209]

In [0]:
# Since I manipulated these data frames earlier in the notebook, 
# I just did a clean re-load of those data frames from the original
# file here.

orders = pd.read_csv("orders.csv")
orders = orders.drop(["user_id", "eval_set", "order_number","order_dow","days_since_prior_order"], axis = 1)
order_products_prior = pd.read_csv("order_products__prior.csv")
order_products_train = pd.read_csv("order_products__train.csv")
order_products = pd.concat([order_products_prior, order_products_train])
order_products = order_products.drop(["add_to_cart_order","reordered"], axis = 1)
products = pd.read_csv("products.csv")
products = products.drop(["aisle_id", "department_id"], axis = 1)

In [0]:
# Now I use the list I generated from the value_counts
# of product_ids earlier to select out only the information corresponding
# to products that have been ordered 2900+ times.

order_products_subs = order_products[order_products["product_id"].isin(list_of_ids)]

In [85]:
product_subs = products[products["product_id"].isin(list_of_ids)]
print(order_products_subs.shape)
product_subs.shape

(22450886, 2)


(2051, 2)

In [0]:
# After painstakingly trimming the data set, it's time to merge.

orders_and_products = pd.merge(order_products_subs, product_subs, on = "product_id")

In [87]:
orders_and_products.head()

,order_id,product_id,product_name
0,2,33120,Organic Egg Whites
1,26,33120,Organic Egg Whites
2,120,33120,Organic Egg Whites
3,327,33120,Organic Egg Whites
4,390,33120,Organic Egg Whites


In [0]:
# Now we need to associate this with the hours of the day, found in the "orders" data.

final_df = pd.merge(orders_and_products, orders, on = "order_id")

In [89]:
final_df.head()

,order_id,product_id,product_name,order_hour_of_day
0,2,33120,Organic Egg Whites,9
1,2,28985,Michigan Organic Kale,9
2,2,9327,Garlic Powder,9
3,2,17794,Carrots,9
4,26,33120,Organic Egg Whites,16


In [0]:
# Finally, we have the table we need. Now we group by 
# the product name and produce the hour of day; finally,
# we sort by the data.

grouped = final_df.pivot_table(values = "order_hour_of_day", index = "product_name")
grouped = grouped.sort_values(by = "order_hour_of_day", ascending = False)

In [95]:
grouped.head(25)

,order_hour_of_day
product_name,
Half Baked Frozen Yogurt,14.526986
The Tonight Dough™ Ice Cream,14.526299
Americone Dream® Ice Cream,14.394912
Half Baked® Ice Cream,14.388942
Phish Food® Ice Cream,14.373308
Milk and Cookies Ice Cream,14.369711
Sicilian Pistachio,14.306421
Chocolate Peanut Butter Cup Gelato,14.303480
Birthday Cake Light Ice Cream,14.292018


## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)